## Sky, IBL, and PBR

In this notebook we will see how to render a sky using the paper 
* "A Scalable and Production Ready Sky and Atmosphere Rendering Technique" by Sébastien Hillaire (2020).

Using the shadertoy implementation from Andrew Helmer.

Then we will see how we can use that sky to create create a diffuse and specular ibl textures using importance sampling. Thanks to Thorsten Thormählen lectures at https://www.uni-marburg.de/en/fb12/research-groups/grafikmultimedia/lectures/graphics 

And we will mix it with pbr lighting from Joey de Vries learnOpenGL website https://learnopengl.com/PBR/Lighting.

### Import and create instance

In [1]:
import ipywebgl
import numpy as np

In [2]:
w = ipywebgl.GLViewer()
w.clear_color(.8, .8, .8 ,1)
w.clear()
w.disable(depth_test=True)
w.execute_commands(execute_once=True)

### Create a simple screen quad

In [3]:
screen_vbo = w.create_buffer_ext(
    #x y u v
    src_data=np.array(
      [-1, 1, 0, 1,
        -1, -1, 0, 0,
        1, -1, 1, 0,
        -1, 1, 0, 1,
        1, -1, 1, 0,
        1, 1, 1, 1], dtype=np.float32).flatten()
)

screen_vao = w.create_vertex_array_ext(
    None,
    [
        (screen_vbo, '2f32 2f32', 0, 1),
    ]
)

### Sky

We will use and modify the implementation from shadertoy from Andrew Helmer.
https://www.shadertoy.com/view/slSXRW

It has several stage, first we have a bunch of common functions and constants used be all the shaders

In [4]:
common_pixel_shader = """#version 300 es
precision highp float;

const float PI = 3.14159265358;

uniform float iTime;
const vec2 iResolution = vec2(640.0, 480.0);


// Units are in megameters.
const float groundRadiusMM = 6.360;
const float atmosphereRadiusMM = 6.460;

// 200M above the ground.
const vec3 viewPos = vec3(0.0, groundRadiusMM + 0.0002, 0.0);

const vec2 tLUTRes = vec2(256.0, 64.0);
const vec2 msLUTRes = vec2(32.0, 32.0);
// Doubled the vertical skyLUT res from the paper, looks way
// better for sunrise.
const vec2 skyLUTRes = vec2(200.0, 200.0);

const vec3 groundAlbedo = vec3(0.2);

// These are per megameter.
const vec3 rayleighScatteringBase = vec3(5.802, 13.558, 33.1);
const float rayleighAbsorptionBase = 0.0;

const float mieScatteringBase = 3.996;
const float mieAbsorptionBase = 4.4;

const vec3 ozoneAbsorptionBase = vec3(0.650, 1.881, .085);

/*
 * Animates the sun movement.
 */
float getSunAltitude(float time)
{
    const float periodSec = 120.0;
    const float halfPeriod = periodSec / 2.0;
    const float sunriseShift = 0.1;
    float cyclePoint = (1.0 - abs((mod(time,periodSec)-halfPeriod)/halfPeriod));
    cyclePoint = (cyclePoint*(1.0+sunriseShift))-sunriseShift;
    return (0.5*PI)*cyclePoint;
}
vec3 getSunDir(float time)
{
    float altitude = getSunAltitude(time);
    return normalize(vec3(0.0, sin(altitude), -cos(altitude)));
}

float getMiePhase(float cosTheta) {
    const float g = 0.8;
    const float scale = 3.0/(8.0*PI);
    
    float num = (1.0-g*g)*(1.0+cosTheta*cosTheta);
    float denom = (2.0+g*g)*pow((1.0 + g*g - 2.0*g*cosTheta), 1.5);
    
    return scale*num/denom;
}

float getRayleighPhase(float cosTheta) {
    const float k = 3.0/(16.0*PI);
    return k*(1.0+cosTheta*cosTheta);
}

void getScatteringValues(vec3 pos, 
                         out vec3 rayleighScattering, 
                         out float mieScattering,
                         out vec3 extinction) {
    float altitudeKM = (length(pos)-groundRadiusMM)*1000.0;
    // Note: Paper gets these switched up.
    float rayleighDensity = exp(-altitudeKM/8.0);
    float mieDensity = exp(-altitudeKM/1.2);
    
    rayleighScattering = rayleighScatteringBase*rayleighDensity;
    float rayleighAbsorption = rayleighAbsorptionBase*rayleighDensity;
    
    mieScattering = mieScatteringBase*mieDensity;
    float mieAbsorption = mieAbsorptionBase*mieDensity;
    
    vec3 ozoneAbsorption = ozoneAbsorptionBase*max(0.0, 1.0 - abs(altitudeKM-25.0)/15.0);
    
    extinction = rayleighScattering + rayleighAbsorption + mieScattering + mieAbsorption + ozoneAbsorption;
}

float safeacos(const float x) {
    return acos(clamp(x, -1.0, 1.0));
}

// From https://gamedev.stackexchange.com/questions/96459/fast-ray-sphere-collision-code.
float rayIntersectSphere(vec3 ro, vec3 rd, float rad) {
    float b = dot(ro, rd);
    float c = dot(ro, ro) - rad*rad;
    if (c > 0.0f && b > 0.0) return -1.0;
    float discr = b*b - c;
    if (discr < 0.0) return -1.0;
    // Special case: inside sphere, use far discriminant
    if (discr > b*b) return (-b + sqrt(discr));
    return -b - sqrt(discr);
}

/*
 * Same parameterization here.
 */
vec3 getValFromTLUT(sampler2D tex, vec2 bufferRes, vec3 pos, vec3 sunDir) {
    float height = length(pos);
    vec3 up = pos / height;
	float sunCosZenithAngle = dot(sunDir, up);
    vec2 uv = vec2(tLUTRes.x*clamp(0.5 + 0.5*sunCosZenithAngle, 0.0, 1.0),
                   tLUTRes.y*max(0.0, min(1.0, (height - groundRadiusMM)/(atmosphereRadiusMM - groundRadiusMM))));
    uv /= bufferRes;
    return texture(tex, uv).rgb;
}
vec3 getValFromMultiScattLUT(sampler2D tex, vec2 bufferRes, vec3 pos, vec3 sunDir) {
    float height = length(pos);
    vec3 up = pos / height;
	float sunCosZenithAngle = dot(sunDir, up);
    vec2 uv = vec2(msLUTRes.x*clamp(0.5 + 0.5*sunCosZenithAngle, 0.0, 1.0),
                   msLUTRes.y*max(0.0, min(1.0, (height - groundRadiusMM)/(atmosphereRadiusMM - groundRadiusMM))));
    uv /= bufferRes;
    return texture(tex, uv).rgb;
}
"""

#### Create the transmittance program
Generates the Transmittance LUT. Each pixel coordinate corresponds to a height and sun zenith angle, and the value is the transmittance from that point to sun, through the atmosphere.

In [5]:
transmittance_prog = w.create_program_ext(
"""#version 300 es

in vec2 in_vert;

void main() {
    gl_Position = vec4(in_vert, 0, 1);
}
"""
,
common_pixel_shader + """
// Buffer A generates the Transmittance LUT. Each pixel coordinate corresponds to a height and sun zenith angle, and
// the value is the transmittance from that point to sun, through the atmosphere.
const float sunTransmittanceSteps = 40.0;

vec3 getSunTransmittance(vec3 pos, vec3 sunDir) {
    if (rayIntersectSphere(pos, sunDir, groundRadiusMM) > 0.0) {
        return vec3(0.0);
    }
    
    float atmoDist = rayIntersectSphere(pos, sunDir, atmosphereRadiusMM);
    float t = 0.0;
    
    vec3 transmittance = vec3(1.0);
    for (float i = 0.0; i < sunTransmittanceSteps; i += 1.0) {
        float newT = ((i + 0.3)/sunTransmittanceSteps)*atmoDist;
        float dt = newT - t;
        t = newT;
        
        vec3 newPos = pos + t*sunDir;
        
        vec3 rayleighScattering, extinction;
        float mieScattering;
        getScatteringValues(newPos, rayleighScattering, mieScattering, extinction);
        
        transmittance *= exp(-dt*extinction);
    }
    return transmittance;
}

out vec4 fragColor;

void main()
{
    float u = gl_FragCoord.x/tLUTRes.x;
    float v = gl_FragCoord.y/tLUTRes.y;
    
    float sunCosTheta = 2.0*u - 1.0;
    float sunTheta = safeacos(sunCosTheta);
    float height = mix(groundRadiusMM, atmosphereRadiusMM, v);
    
    vec3 pos = vec3(0.0, height, 0.0); 
    vec3 sunDir = normalize(vec3(0.0, sunCosTheta, -sin(sunTheta)));
    
    fragColor = vec4(getSunTransmittance(pos, sunDir), 1.0);
}
""",
{
    'in_vert' : 0,
})

#### Create the associated buffer and texture

we create a 256 by 64 buffer that match the constants in the shader.  We bind it to the texture id 0

In [6]:
transmittance_buffer = w.create_framebuffer()
w.bind_framebuffer('FRAMEBUFFER', transmittance_buffer)

transmittance_lut = w.create_texture()
w.active_texture(0)
w.bind_texture('TEXTURE_2D', transmittance_lut)
w.tex_parameter('TEXTURE_2D', 'TEXTURE_MAG_FILTER', 'LINEAR')
w.tex_parameter('TEXTURE_2D', 'TEXTURE_MIN_FILTER', 'LINEAR')
w.tex_parameter('TEXTURE_2D', 'TEXTURE_WRAP_S', 'CLAMP_TO_EDGE')
w.tex_parameter('TEXTURE_2D', 'TEXTURE_WRAP_T', 'CLAMP_TO_EDGE')
w.tex_storage_2d('TEXTURE_2D', 1, 'RGBA16F', 256, 64)
w.framebuffer_texture_2d('FRAMEBUFFER', 'COLOR_ATTACHMENT0', 'TEXTURE_2D', transmittance_lut, 0)

w.bind_framebuffer('FRAMEBUFFER', None)
w.execute_commands(execute_once=True)

#### Create the scattering program
The multiple-scattering LUT. Each pixel coordinate corresponds to a height and sun zenith angle, and the value is the multiple scattering approximation (Psi_ms from the paper, Eq. 10).

In [7]:
scattering_prog = w.create_program_ext(
"""#version 300 es

in vec2 in_vert;

void main() {
    gl_Position = vec4(in_vert, 0, 1);
}
"""
,
common_pixel_shader + """
// Buffer B is the multiple-scattering LUT. Each pixel coordinate corresponds to a height and sun zenith angle, and
// the value is the multiple scattering approximation (Psi_ms from the paper, Eq. 10).

uniform sampler2D u_transmittance;

const float mulScattSteps = 20.0;
const int sqrtSamples = 8;

vec3 getSphericalDir(float theta, float phi) {
     float cosPhi = cos(phi);
     float sinPhi = sin(phi);
     float cosTheta = cos(theta);
     float sinTheta = sin(theta);
     return vec3(sinPhi*sinTheta, cosPhi, sinPhi*cosTheta);
}

// Calculates Equation (5) and (7) from the paper.
void getMulScattValues(vec3 pos, vec3 sunDir, out vec3 lumTotal, out vec3 fms) {
    lumTotal = vec3(0.0);
    fms = vec3(0.0);
    
    float invSamples = 1.0/float(sqrtSamples*sqrtSamples);
    for (int i = 0; i < sqrtSamples; i++) {
        for (int j = 0; j < sqrtSamples; j++) {
            // This integral is symmetric about theta = 0 (or theta = PI), so we
            // only need to integrate from zero to PI, not zero to 2*PI.
            float theta = PI * (float(i) + 0.5) / float(sqrtSamples);
            float phi = safeacos(1.0 - 2.0*(float(j) + 0.5) / float(sqrtSamples));
            vec3 rayDir = getSphericalDir(theta, phi);
            
            float atmoDist = rayIntersectSphere(pos, rayDir, atmosphereRadiusMM);
            float groundDist = rayIntersectSphere(pos, rayDir, groundRadiusMM);
            float tMax = atmoDist;
            if (groundDist > 0.0) {
                tMax = groundDist;
            }
            
            float cosTheta = dot(rayDir, sunDir);
    
            float miePhaseValue = getMiePhase(cosTheta);
            float rayleighPhaseValue = getRayleighPhase(-cosTheta);
            
            vec3 lum = vec3(0.0), lumFactor = vec3(0.0), transmittance = vec3(1.0);
            float t = 0.0;
            for (float stepI = 0.0; stepI < mulScattSteps; stepI += 1.0) {
                float newT = ((stepI + 0.3)/mulScattSteps)*tMax;
                float dt = newT - t;
                t = newT;

                vec3 newPos = pos + t*rayDir;

                vec3 rayleighScattering, extinction;
                float mieScattering;
                getScatteringValues(newPos, rayleighScattering, mieScattering, extinction);

                vec3 sampleTransmittance = exp(-dt*extinction);
                
                // Integrate within each segment.
                vec3 scatteringNoPhase = rayleighScattering + mieScattering;
                vec3 scatteringF = (scatteringNoPhase - scatteringNoPhase * sampleTransmittance) / extinction;
                lumFactor += transmittance*scatteringF;
                
                // This is slightly different from the paper, but I think the paper has a mistake?
                // In equation (6), I think S(x,w_s) should be S(x-tv,w_s).
                vec3 sunTransmittance = getValFromTLUT(u_transmittance, tLUTRes, newPos, sunDir);

                vec3 rayleighInScattering = rayleighScattering*rayleighPhaseValue;
                float mieInScattering = mieScattering*miePhaseValue;
                vec3 inScattering = (rayleighInScattering + mieInScattering)*sunTransmittance;

                // Integrated scattering within path segment.
                vec3 scatteringIntegral = (inScattering - inScattering * sampleTransmittance) / extinction;

                lum += scatteringIntegral*transmittance;
                transmittance *= sampleTransmittance;
            }
            
            if (groundDist > 0.0) {
                vec3 hitPos = pos + groundDist*rayDir;
                if (dot(pos, sunDir) > 0.0) {
                    hitPos = normalize(hitPos)*groundRadiusMM;
                    lum += transmittance*groundAlbedo*getValFromTLUT(u_transmittance, tLUTRes, hitPos, sunDir);
                }
            }
            
            fms += lumFactor*invSamples;
            lumTotal += lum*invSamples;
        }
    }
}

out vec4 fragColor;

void main()
{
    float u = clamp(gl_FragCoord.x, 0.0, msLUTRes.x-1.0)/msLUTRes.x;
    float v = clamp(gl_FragCoord.y, 0.0, msLUTRes.y-1.0)/msLUTRes.y;
    
    float sunCosTheta = 2.0*u - 1.0;
    float sunTheta = safeacos(sunCosTheta);
    float height = mix(groundRadiusMM, atmosphereRadiusMM, v);
    
    vec3 pos = vec3(0.0, height, 0.0); 
    vec3 sunDir = normalize(vec3(0.0, sunCosTheta, -sin(sunTheta)));
    
    vec3 lum, f_ms;
    getMulScattValues(pos, sunDir, lum, f_ms);
    
    // Equation 10 from the paper.
    vec3 psi = lum  / (1.0 - f_ms); 
    fragColor = vec4(psi, 1.0);
}
""",
{
    'in_vert' : 0,
})

#### Create the assiociated scattering buffer

This is a small 32 by 32 buffer.  We bind it to the texture id 1

In [8]:
scattering_buffer = w.create_framebuffer()
w.bind_framebuffer('FRAMEBUFFER', scattering_buffer)

scattering_lut = w.create_texture()
w.active_texture(1)
w.bind_texture('TEXTURE_2D', scattering_lut)
w.tex_parameter('TEXTURE_2D', 'TEXTURE_MAG_FILTER', 'LINEAR')
w.tex_parameter('TEXTURE_2D', 'TEXTURE_MIN_FILTER', 'LINEAR')
w.tex_parameter('TEXTURE_2D', 'TEXTURE_WRAP_S', 'CLAMP_TO_EDGE')
w.tex_parameter('TEXTURE_2D', 'TEXTURE_WRAP_T', 'CLAMP_TO_EDGE')
w.tex_storage_2d('TEXTURE_2D', 1, 'RGBA16F', 32, 32)
w.framebuffer_texture_2d('FRAMEBUFFER', 'COLOR_ATTACHMENT0', 'TEXTURE_2D', scattering_lut, 0)

w.bind_framebuffer('FRAMEBUFFER', None)
w.execute_commands(execute_once=True)

#### Create the Skyview program
Calculates the actual sky-view! It's a lat-long map (or maybe altitude-azimuth is the better term), but the latitude/altitude is non-linear to get more resolution near the horizon.

In [9]:
skyview_prog = w.create_program_ext(
"""#version 300 es

in vec2 in_vert;

void main() {
    gl_Position = vec4(in_vert, 0, 1);
}
"""
,
common_pixel_shader + """
// Buffer C calculates the actual sky-view! It's a lat-long map (or maybe altitude-azimuth is the better term),
// but the latitude/altitude is non-linear to get more resolution near the horizon.

uniform sampler2D u_transmittance;
uniform sampler2D u_scattering;

const int numScatteringSteps = 32;
vec3 raymarchScattering(vec3 pos, 
                              vec3 rayDir, 
                              vec3 sunDir,
                              float tMax,
                              float numSteps) {
    float cosTheta = dot(rayDir, sunDir);
    
	float miePhaseValue = getMiePhase(cosTheta);
	float rayleighPhaseValue = getRayleighPhase(-cosTheta);
    
    vec3 lum = vec3(0.0);
    vec3 transmittance = vec3(1.0);
    float t = 0.0;
    for (float i = 0.0; i < numSteps; i += 1.0) {
        float newT = ((i + 0.3)/numSteps)*tMax;
        float dt = newT - t;
        t = newT;
        
        vec3 newPos = pos + t*rayDir;
        
        vec3 rayleighScattering, extinction;
        float mieScattering;
        getScatteringValues(newPos, rayleighScattering, mieScattering, extinction);
        
        vec3 sampleTransmittance = exp(-dt*extinction);

        vec3 sunTransmittance = getValFromTLUT(u_transmittance, tLUTRes.xy, newPos, sunDir);
        vec3 psiMS = getValFromMultiScattLUT(u_scattering, msLUTRes.xy, newPos, sunDir);
        
        vec3 rayleighInScattering = rayleighScattering*(rayleighPhaseValue*sunTransmittance + psiMS);
        vec3 mieInScattering = mieScattering*(miePhaseValue*sunTransmittance + psiMS);
        vec3 inScattering = (rayleighInScattering + mieInScattering);

        // Integrated scattering within path segment.
        vec3 scatteringIntegral = (inScattering - inScattering * sampleTransmittance) / extinction;

        lum += scatteringIntegral*transmittance;
        
        transmittance *= sampleTransmittance;
    }
    return lum;
}
out vec4 fragColor;
void main()
{
    float u = clamp(gl_FragCoord.x, 0.0, skyLUTRes.x-1.0)/skyLUTRes.x;
    float v = clamp(gl_FragCoord.y, 0.0, skyLUTRes.y-1.0)/skyLUTRes.y;
    
    float azimuthAngle = (u - 0.5)*2.0*PI;
    // Non-linear mapping of altitude. See Section 5.3 of the paper.
    float adjV;
    if (v < 0.5) {
		float coord = 1.0 - 2.0*v;
		adjV = -coord*coord;
	} else {
		float coord = v*2.0 - 1.0;
		adjV = coord*coord;
	}
    
    float height = length(viewPos);
    vec3 up = viewPos / height;
    float horizonAngle = safeacos(sqrt(height * height - groundRadiusMM * groundRadiusMM) / height) - 0.5*PI;
    float altitudeAngle = adjV*0.5*PI - horizonAngle;
    
    float cosAltitude = cos(altitudeAngle);
    vec3 rayDir = vec3(cosAltitude*sin(azimuthAngle), sin(altitudeAngle), -cosAltitude*cos(azimuthAngle));
    
    float sunAltitude = (0.5*PI) - acos(dot(getSunDir(iTime), up));
    vec3 sunDir = vec3(0.0, sin(sunAltitude), -cos(sunAltitude));
    
    float atmoDist = rayIntersectSphere(viewPos, rayDir, atmosphereRadiusMM);
    float groundDist = rayIntersectSphere(viewPos, rayDir, groundRadiusMM);
    float tMax = (groundDist < 0.0) ? atmoDist : groundDist;
    vec3 lum = raymarchScattering(viewPos, rayDir, sunDir, tMax, float(numScatteringSteps));
    fragColor = vec4(lum, 1.0);
}
""",
{
    'in_vert' : 0,
})


#### Create the assicated skyview buffer

A 200 by 200 buffer bound to the texture id 2

In [10]:
skyview_buffer = w.create_framebuffer()
w.bind_framebuffer('FRAMEBUFFER', skyview_buffer)

skyview_tex = w.create_texture()
w.active_texture(2)
w.bind_texture('TEXTURE_2D', skyview_tex)
w.tex_parameter('TEXTURE_2D', 'TEXTURE_MAG_FILTER', 'LINEAR')
w.tex_parameter('TEXTURE_2D', 'TEXTURE_MIN_FILTER', 'LINEAR')
w.tex_parameter('TEXTURE_2D', 'TEXTURE_WRAP_S', 'CLAMP_TO_EDGE')
w.tex_parameter('TEXTURE_2D', 'TEXTURE_WRAP_T', 'CLAMP_TO_EDGE')
w.tex_storage_2d('TEXTURE_2D', 1, 'RGBA16F', 200, 200)
w.framebuffer_texture_2d('FRAMEBUFFER', 'COLOR_ATTACHMENT0', 'TEXTURE_2D', skyview_tex, 0)

w.bind_framebuffer('FRAMEBUFFER', None)
w.execute_commands(execute_once=True)

#### And finally the Sky rendering program

This is the program that can render the sky in the background of our scene.

First here we keep all the common functions used to render, so we can re-use them when doing our IBL renders.

In [11]:
render_sky_functions = """
/*
 * Partial implementation of
 *    "A Scalable and Production Ready Sky and Atmosphere Rendering Technique"
 *    by Sébastien Hillaire (2020).
 * Very much referenced and copied Sébastien's provided code: 
 *    https://github.com/sebh/UnrealEngineSkyAtmosphere
 *
 * This basically implements the generation of a sky-view LUT, so it doesn't
 * include aerial perspective. It only works for views inside the atmosphere,
 * because the code assumes that the ray-marching starts at the camera position.
 * For a planetary view you'd want to check that and you might march from, e.g.
 * the edge of the atmosphere to the ground (rather than the camera position
 * to either the ground or edge of the atmosphere).
 *
 * Also want to cite: 
 *    https://www.shadertoy.com/view/tdSXzD
 * Used the jodieReinhardTonemap from there, but that also made
 * me realize that the paper switched the Mie and Rayleigh height densities
 * (which was confirmed after reading Sébastien's code more closely).
 */

/*
 * Final output basically looks up the value from the skyLUT, and then adds a sun on top,
 * does some tonemapping.
 */
 
uniform sampler2D u_transmittance;
uniform sampler2D u_skyview;
 
vec3 getValFromSkyLUT(vec3 rayDir, vec3 sunDir) {
    float height = length(viewPos);
    vec3 up = viewPos / height;
    
    float horizonAngle = safeacos(sqrt(height * height - groundRadiusMM * groundRadiusMM) / height);
    float altitudeAngle = horizonAngle - acos(dot(rayDir, up)); // Between -PI/2 and PI/2
    float azimuthAngle; // Between 0 and 2*PI
    if (abs(altitudeAngle) > (0.5*PI - .0001)) {
        // Looking nearly straight up or down.
        azimuthAngle = 0.0;
    } else {
        vec3 right = cross(sunDir, up);
        vec3 forward = cross(up, right);
        
        vec3 projectedDir = normalize(rayDir - up*(dot(rayDir, up)));
        float sinTheta = dot(projectedDir, right);
        float cosTheta = dot(projectedDir, forward);
        azimuthAngle = atan(sinTheta, cosTheta) + PI;
    }
    
    // Non-linear mapping of altitude angle. See Section 5.3 of the paper.
    float v = 0.5 + 0.5*sign(altitudeAngle)*sqrt(abs(altitudeAngle)*2.0/PI);
    vec2 uv = vec2(azimuthAngle / (2.0*PI), v);
   // uv *= skyLUTRes;
   // uv /= iChannelResolution[1].xy;
    
    return texture(u_skyview, uv).rgb;
}

vec3 jodieReinhardTonemap(vec3 c){
    // From: https://www.shadertoy.com/view/tdSXzD
    float l = dot(c, vec3(0.2126, 0.7152, 0.0722));
    vec3 tc = c / (c + 1.0);
    return mix(c / (l + 1.0), tc, tc);
}

vec3 sunWithBloom(vec3 rayDir, vec3 sunDir) {
    const float sunSolidAngle = 0.53*PI/180.0;
    const float minSunCosTheta = cos(sunSolidAngle);

    float cosTheta = dot(rayDir, sunDir);
    if (cosTheta >= minSunCosTheta) return vec3(1.0);
    
    float offset = minSunCosTheta - cosTheta;
    float gaussianBloom = exp(-offset*50000.0)*0.5;
    float invBloom = 1.0/(0.02 + offset*300.0)*0.01;
    return vec3(gaussianBloom+invBloom);
}

vec3 createRay(vec2 px, mat4 PInv, mat4 VInv)
{
  
    // convert pixel to NDS
    // [0,1] -> [-1,1]
    vec2 pxNDS = px*2. - 1.;

    // choose an arbitrary point in the viewing volume
    // z = -1 equals a point on the near plane, i.e. the screen
    vec3 pointNDS = vec3(pxNDS, -1.);

    // as this is in homogenous space, add the last homogenous coordinate
    vec4 pointNDSH = vec4(pointNDS, 1.0);
    // transform by inverse projection to get the point in view space
    vec4 dirEye = PInv * pointNDSH;

    // since the camera is at the origin in view space by definition,
    // the current point is already the correct direction 
    // (dir(0,P) = P - 0 = P as a direction, an infinite point,
    // the homogenous component becomes 0 the scaling done by the 
    // w-division is not of interest, as the direction in xyz will 
    // stay the same and we can just normalize it later
    dirEye.w = 0.;

    // compute world ray direction by multiplying the inverse view matrix
    vec3 dirWorld = (VInv * dirEye).xyz;

    // now normalize direction
    return normalize(dirWorld); 
}

"""

And the final program it self.

In [12]:
final_prog = w.create_program_ext(
"""#version 300 es

in vec2 in_vert;

void main() {
    gl_Position = vec4(in_vert, 0, 1);
}
"""
,
common_pixel_shader + render_sky_functions + """
out vec4 fragColor;

layout(std140) uniform ViewBlock
{
    mat4 u_cameraMatrix;          //the camera matrix in world space
    mat4 u_viewMatrix;            //the inverse of the camera matrix
    mat4 u_projectionMatrix;      //the projection matrix
    mat4 u_viewProjectionMatrix;  //the projection * view matrix
};

void main()
{
    vec3 sunDir = getSunDir(iTime);
    vec3 rayDir = createRay((gl_FragCoord.xy /iResolution), inverse(u_projectionMatrix), u_cameraMatrix);
    
    vec3 lum = getValFromSkyLUT(rayDir, sunDir);

    // Bloom should be added at the end, but this is subtle and works well.
    vec3 sunLum = sunWithBloom(rayDir, sunDir);
    // Use smoothstep to limit the effect, so it drops off to actual zero.
    sunLum = smoothstep(0.002, 1.0, sunLum);
    if (length(sunLum) > 0.0) {
        if (rayIntersectSphere(viewPos, rayDir, groundRadiusMM) >= 0.0) {
            sunLum *= 0.0;
        } else {
            // If the sun value is applied to this pixel, we need to calculate the transmittance to obscure it.
            sunLum *= getValFromTLUT(u_transmittance, tLUTRes.xy, viewPos, sunDir);
        }
    }
    lum += sunLum;
    
    // Tonemapping and gamma. Super ad-hoc, probably a better way to do this.
    lum *= 20.0;
    lum = pow(lum, vec3(1.3));
    lum /= (smoothstep(0.0, 0.2, clamp(sunDir.y, 0.0, 1.0))*2.0 + 0.15);
    
    lum = jodieReinhardTonemap(lum);
    
    lum = pow(lum, vec3(1.0/2.2));
    
    fragColor = vec4(lum,1.0);
}
""",
{
    'in_vert' : 0,
})

From here we can actually render the sky in the scene.

## IBL and importance sampling

From here we are in the process of reading the sky texture to convert it into a equirectangular diffuse irradiance and specular irradiance maps.

#### Diffuse IBL

In [13]:
diffuse_ibl_buffer = w.create_framebuffer()
w.bind_framebuffer('FRAMEBUFFER', diffuse_ibl_buffer)

diffuse_ibl_tex = w.create_texture()
w.active_texture(3)
w.bind_texture('TEXTURE_2D', diffuse_ibl_tex)
w.tex_parameter('TEXTURE_2D', 'TEXTURE_MAG_FILTER', 'LINEAR')
w.tex_parameter('TEXTURE_2D', 'TEXTURE_MIN_FILTER', 'LINEAR')
w.tex_parameter('TEXTURE_2D', 'TEXTURE_WRAP_S', 'CLAMP_TO_EDGE')
w.tex_parameter('TEXTURE_2D', 'TEXTURE_WRAP_T', 'CLAMP_TO_EDGE')
w.tex_storage_2d('TEXTURE_2D', 1, 'RGBA16F', 128, 64)
w.framebuffer_texture_2d('FRAMEBUFFER', 'COLOR_ATTACHMENT0', 'TEXTURE_2D', diffuse_ibl_tex, 0)

w.bind_framebuffer('FRAMEBUFFER', None)
w.execute_commands(execute_once=True)

In [17]:
diffuse_importance_sampling_prog = w.create_program_ext(
"""#version 300 es

in vec2 in_vert;

void main() {
    gl_Position = vec4(in_vert, 0, 1);
}
"""
,
common_pixel_shader + render_sky_functions + """
out vec4 outColor;
uniform int u_samples;

mat3 getNormalSpace(in vec3 normal) {
   vec3 someVec = vec3(1.0, 0.0, 0.0);
   float dd = dot(someVec, normal);
   vec3 tangent = vec3(0.0, 1.0, 0.0);
   if(abs(dd) > 1e-8) {
     tangent = normalize(cross(someVec, normal));
   }
   vec3 bitangent = cross(normal, tangent);
   return mat3(tangent, bitangent, normal);
}

// from http://holger.dammertz.org/stuff/notes_HammersleyOnHemisphere.html
// Hacker's Delight, Henry S. Warren, 2001
float radicalInverse(uint bits) {
  bits = (bits << 16u) | (bits >> 16u);
  bits = ((bits & 0x55555555u) << 1u) | ((bits & 0xAAAAAAAAu) >> 1u);
  bits = ((bits & 0x33333333u) << 2u) | ((bits & 0xCCCCCCCCu) >> 2u);
  bits = ((bits & 0x0F0F0F0Fu) << 4u) | ((bits & 0xF0F0F0F0u) >> 4u);
  bits = ((bits & 0x00FF00FFu) << 8u) | ((bits & 0xFF00FF00u) >> 8u);
  return float(bits) * 2.3283064365386963e-10; // / 0x100000000
}

vec2 hammersley(uint n, uint N) {
  return vec2(float(n) / float(N), radicalInverse(n));
}

// The origin of the random2 function is probably the paper:
// 'On generating random numbers, with help of y= [(a+x)sin(bx)] mod 1'
// W.J.J. Rey, 22nd European Meeting of Statisticians and the
// 7th Vilnius Conference on Probability Theory and Mathematical Statistics, August 1998
// as discussed here:
// https://stackoverflow.com/questions/12964279/whats-the-origin-of-this-glsl-rand-one-liner
float random2(vec2 n) { 
	return fract(sin(dot(n, vec2(12.9898, 4.1414))) * 43758.5453);
}

vec3 sun(vec3 rayDir, vec3 sunDir) {
    const float sunSolidAngle = 5.0*PI/180.0;
    const float minSunCosTheta = cos(sunSolidAngle);

    float cosTheta = dot(rayDir, sunDir);
    if (cosTheta >= minSunCosTheta) return vec3(1.0);
    
    return vec3(0);
}

vec3 luminosity(vec3 rayDir, vec3 sunDir) {
    return getValFromSkyLUT(rayDir, sunDir);
}

void main() {
    vec3 sunDir = getSunDir(iTime);
  
  vec2 coord = (gl_FragCoord.xy / vec2(128.0, 64.0));
  float thetaN = PI * (1.0-coord.y);
  float phiN = 2.0 * PI * (coord.x) + (PI/2.0);
  vec3 normal = vec3(sin(thetaN) * cos(phiN), cos(thetaN), sin(thetaN) * sin(phiN));
  mat3 normalSpace = getNormalSpace(normal);

  vec3 result = vec3(0.0);

  uint N = uint(u_samples);
  
  float r = random2(coord);
  
  for(uint n = 1u; n <= N; n++) {
      //vec2 p = hammersley(n, N);
      vec2 p = mod(hammersley(n, N) + r, 1.0);
      float theta = asin(sqrt(p.y));
      float phi = 2.0 * PI * p.x;
      vec3 pos = vec3(sin(theta) * cos(phi), sin(theta) * sin(phi), cos(theta));
      vec3 posGlob = normalSpace * pos;
      vec3 radiance = luminosity(posGlob,sunDir).rgb;
      result += radiance;
  }
  result = result / float(u_samples);
  
   result *= 20.0;
    result = pow(result, vec3(1.3));
  
  outColor.rgb = result;
  outColor.a = 1.0;
}

""",
{
    'in_vert' : 0,
})

#### brdf integration

In [18]:
brdf_integration_buffer = w.create_framebuffer()
w.bind_framebuffer('FRAMEBUFFER', brdf_integration_buffer)

brdf_integration_tex = w.create_texture()
w.active_texture(4)
w.bind_texture('TEXTURE_2D', brdf_integration_tex)
w.tex_parameter('TEXTURE_2D', 'TEXTURE_MAG_FILTER', 'LINEAR')
w.tex_parameter('TEXTURE_2D', 'TEXTURE_MIN_FILTER', 'LINEAR')
w.tex_parameter('TEXTURE_2D', 'TEXTURE_WRAP_S', 'CLAMP_TO_EDGE')
w.tex_parameter('TEXTURE_2D', 'TEXTURE_WRAP_T', 'CLAMP_TO_EDGE')
w.tex_storage_2d('TEXTURE_2D', 1, 'RGBA16F', 128, 128)
w.framebuffer_texture_2d('FRAMEBUFFER', 'COLOR_ATTACHMENT0', 'TEXTURE_2D', brdf_integration_tex, 0)

w.bind_framebuffer('FRAMEBUFFER', None)
w.execute_commands(execute_once=True)

In [21]:
brdf_integration_prog = w.create_program_ext(
"""#version 300 es

in vec2 in_vert;

void main() {
    gl_Position = vec4(in_vert, 0, 1);
}
"""
,
"""#version 300 es
precision highp float;

out vec4 outColor;
uniform int u_samples; // description="number of samples" 
const float PI = 3.1415926535897932384626433832795;

// from http://holger.dammertz.org/stuff/notes_HammersleyOnHemisphere.html
// Hacker's Delight, Henry S. Warren, 2001
float radicalInverse(uint bits) {
  bits = (bits << 16u) | (bits >> 16u);
  bits = ((bits & 0x55555555u) << 1u) | ((bits & 0xAAAAAAAAu) >> 1u);
  bits = ((bits & 0x33333333u) << 2u) | ((bits & 0xCCCCCCCCu) >> 2u);
  bits = ((bits & 0x0F0F0F0Fu) << 4u) | ((bits & 0xF0F0F0F0u) >> 4u);
  bits = ((bits & 0x00FF00FFu) << 8u) | ((bits & 0xFF00FF00u) >> 8u);
  return float(bits) * 2.3283064365386963e-10; // / 0x100000000
}

vec2 hammersley(uint n, uint N) {
  return vec2(float(n) / float(N), radicalInverse(n));
}

float G1_GGX_Schlick(float NdotV, float roughness) {
  float r = roughness; // original
  //float r = 0.5 + 0.5 * roughness; // Disney remapping
  float k = (r * r) / 2.0;
  float denom = NdotV * (1.0 - k) + k;
  return NdotV / denom;
}

float G_Smith(float NoV, float NoL, float roughness) {
  float g1_l = G1_GGX_Schlick(NoL, roughness);
  float g1_v = G1_GGX_Schlick(NoV, roughness);
  return g1_l * g1_v;
}

// adapted from "Real Shading in Unreal Engine 4", Brian Karis, Epic Games
// https://cdn2.unrealengine.com/Resources/files/2013SiggraphPresentationsNotes-26915738.pdf
vec2 integrateBRDF(float roughness, float NoV) {
  vec3 V;
  V.x = sqrt(1.0 - NoV * NoV); // sin
  V.y = 0.0;
  V.z = NoV; // cos
  vec2 result = vec2(0.0);
  uint sampleCount = uint(u_samples);
  for(uint n = 1u; n <= sampleCount; n++) {
    vec2 p = hammersley(n, sampleCount);
    float a = roughness * roughness;
    float theta = acos(sqrt((1.0 - p.y) / (1.0 + (a * a - 1.0) * p.y)));
    float phi = 2.0 * PI * p.x;
    // sampled h direction in normal space
    vec3 H = vec3(sin(theta) * cos(phi), sin(theta) * sin(phi), cos(theta));
    vec3 L = 2.0 * dot(V, H) * H - V;

    // because N = vec3(0.0, 0.0, 1.0) follows
    float NoL = clamp(L.z, 0.0, 1.0);
    float NoH = clamp(H.z, 0.0, 1.0);
    float VoH = clamp(dot(V, H), 0.0, 1.0);
    if(NoL > 0.0) {
      float G = G_Smith(NoV, NoL, roughness);
      float G_Vis = G * VoH / (NoH * NoV);
      float Fc = pow(1.0 - VoH, 5.0);
      result.x += (1.0 - Fc) * G_Vis;
      result.y += Fc * G_Vis;
    }
  }
  result = result / float(sampleCount);
  return result;
}

void main() {
  vec2 r = integrateBRDF(gl_FragCoord.y/128.0, gl_FragCoord.x/128.0);
  outColor = vec4(r, 0.0, 1.0);
}

""",
{
    'in_vert' : 0,
})

#### brdf convolution

In [22]:
brdf_tex = w.create_texture()
w.active_texture(5)
w.bind_texture('TEXTURE_2D', brdf_tex)
w.tex_parameter('TEXTURE_2D', 'TEXTURE_MAG_FILTER', 'LINEAR_MIPMAP_LINEAR')
w.tex_parameter('TEXTURE_2D', 'TEXTURE_MIN_FILTER', 'LINEAR_MIPMAP_LINEAR')
w.tex_parameter('TEXTURE_2D', 'TEXTURE_WRAP_S', 'CLAMP_TO_EDGE')
w.tex_parameter('TEXTURE_2D', 'TEXTURE_WRAP_T', 'CLAMP_TO_EDGE')
w.tex_storage_2d('TEXTURE_2D', 6, 'RGBA16F', 512, 256)

brdf_buffers = [0]*6
for i in range(6):
    brdf_buffers[i] = w.create_framebuffer()
    w.bind_framebuffer('FRAMEBUFFER', brdf_buffers[i])
    w.framebuffer_texture_2d('FRAMEBUFFER', 'COLOR_ATTACHMENT0', 'TEXTURE_2D', brdf_tex, i)

w.bind_framebuffer('FRAMEBUFFER', None)
w.execute_commands(execute_once=True)

In [25]:
brdf_importance_sampling_prog = w.create_program_ext(
"""#version 300 es

in vec2 in_vert;

void main() {
    gl_Position = vec4(in_vert, 0, 1);
}
"""
,
common_pixel_shader + render_sky_functions + """
out vec4 outColor;
uniform int u_samples;
uniform float u_roughness; // description="roughness in range [0.0, 1.0]" 
uniform vec2 u_resolution;

mat3 getNormalSpace(in vec3 normal) {
   vec3 someVec = vec3(1.0, 0.0, 0.0);
   float dd = dot(someVec, normal);
   vec3 tangent = vec3(0.0, 1.0, 0.0);
   if(abs(dd) > 1e-8) {
     tangent = normalize(cross(someVec, normal));
   }
   vec3 bitangent = cross(normal, tangent);
   return mat3(tangent, bitangent, normal);
}

// from http://holger.dammertz.org/stuff/notes_HammersleyOnHemisphere.html
// Hacker's Delight, Henry S. Warren, 2001
float radicalInverse(uint bits) {
  bits = (bits << 16u) | (bits >> 16u);
  bits = ((bits & 0x55555555u) << 1u) | ((bits & 0xAAAAAAAAu) >> 1u);
  bits = ((bits & 0x33333333u) << 2u) | ((bits & 0xCCCCCCCCu) >> 2u);
  bits = ((bits & 0x0F0F0F0Fu) << 4u) | ((bits & 0xF0F0F0F0u) >> 4u);
  bits = ((bits & 0x00FF00FFu) << 8u) | ((bits & 0xFF00FF00u) >> 8u);
  return float(bits) * 2.3283064365386963e-10; // / 0x100000000
}

vec2 hammersley(uint n, uint N) {
  return vec2(float(n) / float(N), radicalInverse(n));
}

// The origin of the random2 function is probably the paper:
// 'On generating random numbers, with help of y= [(a+x)sin(bx)] mod 1'
// W.J.J. Rey, 22nd European Meeting of Statisticians and the
// 7th Vilnius Conference on Probability Theory and Mathematical Statistics, August 1998
// as discussed here:
// https://stackoverflow.com/questions/12964279/whats-the-origin-of-this-glsl-rand-one-liner
float random2(vec2 n) { 
	return fract(sin(dot(n, vec2(12.9898, 4.1414))) * 43758.5453);
}

vec3 sun(vec3 rayDir, vec3 sunDir) {
    const float sunSolidAngle = 5.0*PI/180.0;
    const float minSunCosTheta = cos(sunSolidAngle);

    float cosTheta = dot(rayDir, sunDir);
    if (cosTheta >= minSunCosTheta) return vec3(1.0);
    
    return vec3(0);
}

vec3 luminosity(vec3 rayDir, vec3 sunDir) {
    return getValFromSkyLUT(rayDir, sunDir);
}

void main() {
  
  vec2 coord = (gl_FragCoord.xy / u_resolution);
  float thetaN = PI * (1.0-coord.y);
  float phiN = 2.0 * PI * (coord.x) + (PI/2.0);
  vec3 R = vec3(sin(thetaN) * cos(phiN), cos(thetaN), sin(thetaN) * sin(phiN));
  
  vec3 sunDir = getSunDir(iTime);

  vec3 N = R;
  vec3 V = R;
  uint sampleCount = uint(u_samples);
  float r = random2(coord);
  float r2 = random2(coord*10.33f);
  mat3 normalSpace = getNormalSpace(N);
  
  float totalWeight = 0.0;
  vec3 result = vec3(0.0);
  
  for(uint n = 1u; n <= sampleCount; n++) {
    //vec2 p = hammersley(n, sampleCount);
    vec2 p = mod(hammersley(n, sampleCount) + r, 1.0);
    float a = u_roughness * u_roughness;
    float theta = acos(sqrt((1.0 - p.y) / (1.0 + (a * a - 1.0) * p.y)));
    float phi = 2.0 * PI * (p.x + r2*0.1);
    // sampled h direction in normal space
    vec3 Hn = vec3(sin(theta) * cos(phi), sin(theta) * sin(phi), cos(theta));
    // sampled h direction in world space
    vec3 H = normalSpace * Hn;
    vec3 L = 2.0 * dot(V, H) * H - V;
    
    float NoL = max(dot(N, L), 0.0);
    if( NoL > 0.0 ) {
      vec3 radiance = luminosity(L,sunDir).rgb;
      result += radiance * NoL;
      totalWeight +=NoL;
    }
  }
  result = result / totalWeight;
  result *= 20.0;
  result = pow(result, vec3(1.3));
  
  outColor.rgb = result;
  outColor.a = 1.0;
}
""",
{
    'in_vert' : 0,
})

#### Display a grid
program and vao for a grid

In [26]:
grid_program = w.create_program_ext(
"""#version 300 es

//the ViewBlock that is automatically filled by ipywebgl
layout(std140) uniform ViewBlock
{
    mat4 u_cameraMatrix;          //the camera matrix in world space
    mat4 u_viewMatrix;            //the inverse of the camera matrix
    mat4 u_projectionMatrix;      //the projection matrix
    mat4 u_viewProjectionMatrix;  //the projection * view matrix
};

in vec3 in_vert;

void main() {
    gl_Position = u_viewProjectionMatrix * vec4(in_vert, 1.0);
}
"""
,
"""#version 300 es
precision highp float;
out vec4 f_color;
void main() {
    f_color = vec4(0.9, 0.1, 0.1, 1.0);
}
""")

def grid(size, steps):
    u = np.repeat(np.linspace(-size, size, steps), 2)
    v = np.tile([-size, size], steps)
    w = np.zeros(steps * 2)
    return np.concatenate([np.dstack([u, w, v]), np.dstack([v, w, u])], dtype=np.float32)

points = grid(500,40)

grid_vbo = w.create_buffer_ext(
    src_data=points
)

grid_vao = w.create_vertex_array_ext(
    grid_program,
    [
        (grid_vbo, '3f32', 'in_vert'),
    ]
)

#### Sphere
IBL + PBR renderer for a sphere

In [29]:
sphere_prog = w.create_program_ext(
"""#version 300 es

//the ViewBlock that is automatically filled by ipywebgl
layout(std140) uniform ViewBlock
{
    mat4 u_cameraMatrix;          //the camera matrix in world space
    mat4 u_viewMatrix;            //the inverse of the camera matrix
    mat4 u_projectionMatrix;      //the projection matrix
    mat4 u_viewProjectionMatrix;  //the projection * view matrix
};

in vec3 in_vert;
in vec3 in_normal;

out vec4 v_normal;
out vec4 v_view;

void main() {
    v_normal = vec4(in_normal, 0);
    v_view = vec4(u_cameraMatrix[3].xyz - in_vert, 0);
    gl_Position = u_viewProjectionMatrix * vec4(in_vert * 50.0, 1.0);
  }
"""
,
common_pixel_shader + render_sky_functions + """

// material parameters
uniform vec3 u_albedo;
uniform vec3 u_emissive;
uniform float u_metallic;
uniform float u_roughness;
uniform float u_reflectance;
uniform int u_mipCount;

uniform vec3 u_lightdir;

uniform sampler2D specularIBLmap;
uniform sampler2D brdfIntegrationmap;
uniform sampler2D diffuseIBLmap;

in vec4 v_normal;
in vec4 v_view;

out vec4 fragColor;

vec2 directionToSphericalEnvmap(vec3 dir) {
  float s = 1.0 - mod(1.0 / (2.0*PI) * atan(dir.x, dir.z), 1.0);
  float t = 1.0 / (PI) * acos(-dir.y);
  return vec2(s, t);
}

// adapted from "Real Shading in Unreal Engine 4", Brian Karis, Epic Games
// https://cdn2.unrealengine.com/Resources/files/2013SiggraphPresentationsNotes-26915738.pdf
vec3 specularIBL(vec3 F0 , float roughness, vec3 N, vec3 V) {
  float NoV = clamp(dot(N, V), 0.0, 1.0);
  vec3 R = reflect(-V, N);
  vec2 uv = directionToSphericalEnvmap(R);
  vec3 prefilteredColor = textureLod(specularIBLmap, uv, roughness*float(u_mipCount)).rgb;
  vec4 brdfIntegration = texture(brdfIntegrationmap, vec2(NoV, roughness));
  return prefilteredColor * ( F0 * brdfIntegration.x + brdfIntegration.y );
}

vec3 diffuseIBL(vec3 normal) {
  vec2 uv = directionToSphericalEnvmap(normal);
  return texture(diffuseIBLmap, uv).rgb;
}

float DistributionGGX(vec3 N, vec3 H, float roughness)
{
    float a      = roughness*roughness;
    float a2     = a*a;
    float NdotH  = max(dot(N, H), 0.0);
    float NdotH2 = NdotH*NdotH;
	
    float num   = a2;
    float denom = (NdotH2 * (a2 - 1.0) + 1.0);
    denom = PI * denom * denom;
	
    return num / denom;
}

float GeometrySchlickGGX(float NdotV, float roughness)
{
    float r = (roughness + 1.0);
    float k = (r*r) / 8.0;

    float num   = NdotV;
    float denom = NdotV * (1.0 - k) + k;
	
    return num / denom;
}
float GeometrySmith(vec3 N, vec3 V, vec3 L, float roughness)
{
    float NdotV = max(dot(N, V), 0.0);
    float NdotL = max(dot(N, L), 0.0);
    float ggx2  = GeometrySchlickGGX(NdotV, roughness);
    float ggx1  = GeometrySchlickGGX(NdotL, roughness);
	
    return ggx1 * ggx2;
}
vec3 fresnelSchlick(float cosTheta, vec3 F0) {
  return F0 + (1.0 - F0) * pow(1.0 - cosTheta, 5.0);
} 

void main() {
  vec3 normal = normalize(v_normal.xyz);
  vec3 viewDir = normalize(v_view.xyz);
  
    vec3 baseColor = pow(u_albedo, vec3(2.2));
    vec3 emission = pow(u_emissive, vec3(2.2));
    float roughness = u_roughness;
    float metallic = u_metallic;

    // F0 for dielectics in range [0.0, 0.16] 
    // default FO is (0.16 * 0.5^2) = 0.04
    vec3 f0 = vec3(0.16 * (u_reflectance * u_reflectance)); 
    // in case of metals, baseColor contains F0
    f0 = mix(f0, baseColor, metallic);

    // compute diffuse and specular factors
    vec3 F = fresnelSchlick(max(dot(normal, viewDir), 0.0), f0);
    vec3 kS = F;
    vec3 kD = 1.0 - kS;
    kD *= 1.0 - metallic; 
    
    vec3 specular = specularIBL(f0, roughness, normal, viewDir); 
    vec3 diffuse = diffuseIBL(normal);

    // shading front-facing
    vec3 color = emission + kD * baseColor * diffuse + specular;
    
    // calculate sun radiance
    {
        vec3 L = u_lightdir.xyz;
        vec3 H = normalize(viewDir + L);
        vec3 radiance = vec3(2.0);  
        
        if (rayIntersectSphere(viewPos, L, groundRadiusMM) >= 0.0) {
            radiance *= 0.0;
        } else {
            // If the sun value is applied to this pixel, we need to calculate the transmittance to obscure it.
            radiance *= getValFromTLUT(u_transmittance, tLUTRes.xy, viewPos, L);
        }

        // cook-torrance brdf
        float NDF = DistributionGGX(normal, H, roughness);        
        float G   = GeometrySmith(normal, viewDir, L, roughness);      
        vec3 F    = fresnelSchlick(max(dot(H, viewDir), 0.0), f0);       

        vec3 kS = F;
        vec3 kD = vec3(1.0) - kS;
        kD *= 1.0 - metallic;	  

        vec3 numerator    = NDF * G * F;
        float denominator = 4.0 * max(dot(normal, viewDir), 0.0) * max(dot(normal, L), 0.0) + 0.0001;
        vec3 specular     = numerator / denominator;  

        // add to outgoing radiance Lo
        float NdotL = max(dot(normal, L), 0.0);                
        color += (kD * baseColor / PI + specular) * radiance * NdotL;
    }

    
    fragColor.rgb = pow(color, vec3(1.0/2.2));
    fragColor.a = 1.0;
  
}
"""
,
{
    'in_vert' : 0,
    'in_normal' : 1
})
sphere_vbo = w.create_buffer_ext(
    src_data=np.array(
      [[ 0.  ,  0.  ,  1.  ,  0.  ,  0.  ,  1.  ],
       [-0.72, -0.53,  0.45, -0.72, -0.53,  0.45],
       [ 0.28, -0.85,  0.45,  0.28, -0.85,  0.45],
       [ 0.89,  0.  ,  0.45,  0.89,  0.  ,  0.45],
       [ 0.28,  0.85,  0.45,  0.28,  0.85,  0.45],
       [-0.72,  0.53,  0.45, -0.72,  0.53,  0.45],
       [-0.89, -0.  , -0.45, -0.89, -0.  , -0.45],
       [-0.28, -0.85, -0.45, -0.28, -0.85, -0.45],
       [ 0.72, -0.53, -0.45,  0.72, -0.53, -0.45],
       [ 0.72,  0.53, -0.45,  0.72,  0.53, -0.45],
       [-0.28,  0.85, -0.45, -0.28,  0.85, -0.45],
       [-0.  ,  0.  , -1.  , -0.  ,  0.  , -1.  ],
       [ 0.16, -0.5 ,  0.85,  0.16, -0.5 ,  0.85],
       [-0.43, -0.31,  0.85, -0.43, -0.31,  0.85],
       [-0.26, -0.81,  0.53, -0.26, -0.81,  0.53],
       [ 0.53, -0.  ,  0.85,  0.53, -0.  ,  0.85],
       [ 0.69, -0.5 ,  0.53,  0.69, -0.5 ,  0.53],
       [ 0.16,  0.5 ,  0.85,  0.16,  0.5 ,  0.85],
       [ 0.69,  0.5 ,  0.53,  0.69,  0.5 ,  0.53],
       [-0.43,  0.31,  0.85, -0.43,  0.31,  0.85],
       [-0.26,  0.81,  0.53, -0.26,  0.81,  0.53],
       [-0.85, -0.  ,  0.53, -0.85, -0.  ,  0.53],
       [-0.59, -0.81, -0.  , -0.59, -0.81, -0.  ],
       [-0.  , -1.  , -0.  , -0.  , -1.  , -0.  ],
       [ 0.59, -0.81,  0.  ,  0.59, -0.81,  0.  ],
       [ 0.95, -0.31, -0.  ,  0.95, -0.31, -0.  ],
       [ 0.95,  0.31, -0.  ,  0.95,  0.31, -0.  ],
       [ 0.59,  0.81, -0.  ,  0.59,  0.81, -0.  ],
       [-0.  ,  1.  , -0.  , -0.  ,  1.  , -0.  ],
       [-0.59,  0.81, -0.  , -0.59,  0.81, -0.  ],
       [-0.95,  0.31,  0.  , -0.95,  0.31,  0.  ],
       [-0.95, -0.31,  0.  , -0.95, -0.31,  0.  ],
       [-0.69, -0.5 , -0.53, -0.69, -0.5 , -0.53],
       [ 0.26, -0.81, -0.53,  0.26, -0.81, -0.53],
       [ 0.85,  0.  , -0.53,  0.85,  0.  , -0.53],
       [ 0.26,  0.81, -0.53,  0.26,  0.81, -0.53],
       [-0.69,  0.5 , -0.53, -0.69,  0.5 , -0.53],
       [-0.53, -0.  , -0.85, -0.53, -0.  , -0.85],
       [-0.16, -0.5 , -0.85, -0.16, -0.5 , -0.85],
       [ 0.43, -0.31, -0.85,  0.43, -0.31, -0.85],
       [ 0.43,  0.31, -0.85,  0.43,  0.31, -0.85],
       [-0.16,  0.5 , -0.85, -0.16,  0.5 , -0.85]], dtype=np.float32).flatten()
)
indices = np.array(
    [[0, 13, 12], [12, 14, 2], [12, 13, 14], [13, 1, 14], [0, 12, 15], [15, 16, 3], [15, 12, 16], [12, 2, 16],
          [0, 15, 17], [17, 18, 4], [17, 15, 18], [15, 3, 18], [0, 17, 19], [19, 20, 5], [19, 17, 20], [17, 4, 20],
          [0, 19, 13], [13, 21, 1], [13, 19, 21], [19, 5, 21], [1, 22, 14], [14, 23, 2], [14, 22, 23], [22, 7, 23],
          [2, 24, 16], [16, 25, 3], [16, 24, 25], [24, 8, 25], [3, 26, 18], [18, 27, 4], [18, 26, 27], [26, 9, 27],
          [4, 28, 20], [20, 29, 5], [20, 28, 29], [28, 10, 29], [5, 30, 21], [21, 31, 1], [21, 30, 31], [30, 6, 31],
          [1, 31, 22], [22, 32, 7], [22, 31, 32], [31, 6, 32], [2, 23, 24], [24, 33, 8], [24, 23, 33], [23, 7, 33],
          [3, 25, 26], [26, 34, 9], [26, 25, 34], [25, 8, 34], [4, 27, 28], [28, 35, 10], [28, 27, 35], [27, 9, 35],
          [5, 29, 30], [30, 36, 6], [30, 29, 36], [29, 10, 36], [6, 37, 32], [32, 38, 7], [32, 37, 38], [37, 11, 38],
          [7, 38, 33], [33, 39, 8], [33, 38, 39], [38, 11, 39], [8, 39, 34], [34, 40, 9], [34, 39, 40], [39, 11, 40],
          [9, 40, 35], [35, 41, 10], [35, 40, 41], [40, 11, 41], [10, 41, 36], [36, 37, 6], [36, 41, 37], [41, 11, 37]],
        dtype=np.uint8).flatten()

sphere_vao = w.create_vertex_array_ext(
    sphere_prog,
    [
        (sphere_vbo, '3f32 3f32', 'in_vert', 'in_normal'),
    ],
    indices
)

#### Debug program
helper program to render the texture created

In [30]:
show_texture_prog = w.create_program_ext(
"""#version 300 es

in vec2 in_vert;
in vec2 in_coord;
out vec2 v_coord;

void main() {
    gl_Position = vec4(in_vert, 0, 1);
    v_coord = in_coord;
}
"""
,
"""#version 300 es
precision highp float;

uniform sampler2D u_texture;
uniform float u_miplevel;
in vec2 v_coord;
out vec4 color;

void main() {
    //ivec2 size = textureSize(u_texture, 0); //so we can display smaller textures
    //color = vec4(texelFetch(u_texture, ivec2(gl_FragCoord.xy), 0).rgb, 1.0);
    //color = vec4(gl_FragCoord.xyz,1);
    color = vec4(textureLod(u_texture, v_coord, u_miplevel).rgb, 1);
}
""",
{
    'in_vert' : 0,
    'in_coord' : 1,
})

In [31]:
def getSunAltitude(time):
    periodSec = 120.0;
    halfPeriod = periodSec / 2.0
    sunriseShift = 0.1
    cyclePoint = (1.0 - np.abs(((time%periodSec)-halfPeriod)/halfPeriod))
    cyclePoint = (cyclePoint*(1.0+sunriseShift))-sunriseShift
    return (0.5*np.pi)*cyclePoint

def getSunDir(time):
    altitude = getSunAltitude(time);
    return np.array([0.0, np.sin(altitude), -np.cos(altitude)], dtype=np.float32)


## Final render

In [34]:
from ipywidgets import widgets, interact

def render(time, rough, metal, reflectance, debug_texture, debug_mip):
    w.bind_framebuffer('FRAMEBUFFER', transmittance_buffer)
    w.viewport(0, 0, 256, 64)
    w.clear()
    w.use_program(transmittance_prog)
    w.uniform('iTime', np.array([time], dtype=np.float32))
    w.bind_vertex_array(screen_vao)
    w.draw_arrays('TRIANGLES',0, 6)
    w.bind_framebuffer('FRAMEBUFFER', None)
    w.viewport(0, 0, w.width, w.height)

    w.bind_framebuffer('FRAMEBUFFER', scattering_buffer)
    w.viewport(0, 0, 32, 32)
    w.clear()
    w.use_program(scattering_prog)
    w.uniform('iTime', np.array([time], dtype=np.float32))
    w.uniform('u_transmittance', np.array([0], dtype=np.int32))
    w.bind_vertex_array(screen_vao)
    w.draw_arrays('TRIANGLES',0, 6)
    w.bind_framebuffer('FRAMEBUFFER', None)
    w.viewport(0, 0, w.width, w.height)

    w.bind_framebuffer('FRAMEBUFFER', skyview_buffer)
    w.viewport(0, 0, 200, 200)
    w.clear()
    w.use_program(skyview_prog)
    w.uniform('iTime', np.array([time], dtype=np.float32))
    w.uniform('u_transmittance', np.array([0], dtype=np.int32))
    w.uniform('u_scattering', np.array([1], dtype=np.int32))
    w.bind_vertex_array(screen_vao)
    w.draw_arrays('TRIANGLES',0, 6)
    w.bind_framebuffer('FRAMEBUFFER', None)
    w.viewport(0, 0, w.width, w.height)
    
    w.bind_framebuffer('FRAMEBUFFER', diffuse_ibl_buffer)
    w.viewport(0, 0, 128, 64)
    w.clear()
    w.use_program(diffuse_importance_sampling_prog)
    w.uniform('iTime', np.array([time], dtype=np.float32))
    w.uniform('u_samples', np.array([2048], dtype=np.int32))
    w.uniform('u_transmittance', np.array([0], dtype=np.int32))
    w.uniform('u_skyview', np.array([2], dtype=np.int32))
    w.bind_vertex_array(screen_vao)
    w.draw_arrays('TRIANGLES',0, 6)
    w.bind_framebuffer('FRAMEBUFFER', None)
    w.viewport(0, 0, w.width, w.height)
    
    w.bind_framebuffer('FRAMEBUFFER', brdf_integration_buffer)
    w.viewport(0, 0, 128, 128)
    w.clear()
    w.use_program(brdf_integration_prog)
    w.uniform('u_samples', np.array([128], dtype=np.int32))
    w.bind_vertex_array(screen_vao)
    w.draw_arrays('TRIANGLES',0, 6)
    w.bind_framebuffer('FRAMEBUFFER', None)
    w.viewport(0, 0, w.width, w.height)
    
    for i in range(6):
        w.bind_framebuffer('FRAMEBUFFER', brdf_buffers[i])
        w.viewport(0, 0, 1024//(2**i), 512//(2**i))
        w.clear()
        w.use_program(brdf_importance_sampling_prog)
        w.uniform('iTime', np.array([time], dtype=np.float32))
        w.uniform('u_transmittance', np.array([0], dtype=np.int32))
        w.uniform('u_skyview', np.array([2], dtype=np.int32))
        w.uniform('u_resolution', np.array([512//(2**i), 256//(2**i)], dtype=np.float32))
        w.uniform('u_samples', np.array([512], dtype=np.int32))
        w.uniform('u_roughness', np.array([0.2*i], dtype=np.float32))
        
        w.bind_vertex_array(screen_vao)
        w.draw_arrays('TRIANGLES',0, 6)
    w.bind_framebuffer('FRAMEBUFFER', None)
    w.viewport(0, 0, w.width, w.height)
    
    # we only need to compute the buffers once when we change the time of day
    w.execute_commands(execute_once=True)

    # draw the actual image
    w.clear()
    w.disable(depth_test=True)
    w.use_program(final_prog)
    w.uniform('iTime', np.array([time], dtype=np.float32))
    w.uniform('u_transmittance', np.array([0], dtype=np.int32))
    w.uniform('u_skyview', np.array([2], dtype=np.int32))
    w.bind_vertex_array(screen_vao)
    w.draw_arrays('TRIANGLES',0, 6)
       
    w.enable(depth_test=True)
    w.use_program(sphere_prog)
    w.uniform('u_albedo', np.array([.9,.9,.9], dtype=np.float32))
    w.uniform('u_metallic', np.array([metal], dtype=np.float32))
    w.uniform('u_roughness', np.array([rough], dtype=np.float32))
    w.uniform('u_reflectance', np.array([reflectance], dtype=np.float32))
    w.uniform('u_mipCount', np.array([6], dtype=np.int32))
    w.uniform('u_lightdir', getSunDir(time))
    w.uniform('u_transmittance', np.array([0], dtype=np.int32))
    w.uniform('diffuseIBLmap', np.array([3], dtype=np.int32))
    w.uniform('brdfIntegrationmap', np.array([4], dtype=np.int32))
    w.uniform('specularIBLmap', np.array([5], dtype=np.int32))
    w.bind_vertex_array(sphere_vao)
    w.draw_elements('TRIANGLES', indices.shape[0], 'UNSIGNED_BYTE', 0)
    
    w.use_program(grid_program)
    w.bind_vertex_array(grid_vao)
    w.draw_arrays('LINES', 0, points.reshape([-1,3]).shape[0])
    
    if debug_texture > 0:
        w.clear()
        w.disable(depth_test=True)
        w.use_program(show_texture_prog)
        w.uniform('u_texture', np.array([debug_texture-1], dtype=np.int32))
        w.uniform('u_miplevel', np.array([debug_mip], dtype=np.float32))
        w.bind_vertex_array(screen_vao)
        w.draw_arrays('TRIANGLES',0, 6)
    
    w.execute_commands()
  
interact(
    render, 
    time=widgets.FloatSlider(min=0, max=60, step=.1, value=9),
    rough=widgets.FloatSlider(min=0, max=1, step=.01, value=.1),
    metal=widgets.FloatSlider(min=0, max=1, step=.01, value=0),
    reflectance=widgets.FloatSlider(min=0, max=1, step=.01, value=.5),
    debug_texture=widgets.IntSlider(min=0, max=6, value=0),
    debug_mip=widgets.IntSlider(min=0, max=5, value=0),
)
w

interactive(children=(FloatSlider(value=9.0, description='time', max=60.0), FloatSlider(value=0.1, description…

GLViewer(camera_pitch=-47.599999999999994, camera_pos=[-54.097238191754215, 129.11720284314038, -118.962350484…